In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationconfig:
    root_dir: Path
    data_path: Path
    horizon: int

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationconfig:
        config = self.config.data_transformation
        horizon = self.params.horizon.h
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationconfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            horizon=horizon
        )
        return data_transformation_config

In [8]:
import os
from src.mlProject.utils import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [9]:
class DataTransformation:
    def __init__(self,config: DataTransformationconfig):
        self.config = config
        
    
        
    def data_windowing(self):
        df = pd.read_csv(self.config.data_path)
        combined_df = pd.DataFrame()
        for s in df['store'].unique():
            for p in df['product'].unique():
                data = df[(df['product'] == p )& (df['store'] == s )].copy()
                for i in range(self.config.horizon):
                    data[f'Y (t+{self.config.horizon-i})'] = data['number_sold'].shift(i)

                for i in range(self.config.horizon,2*self.config.horizon):
                    data[f'X (t-{i-self.config.horizon})'] = data['number_sold'].shift(i)

                data.dropna(inplace=True)
                combined_df = pd.concat([combined_df, data], ignore_index=True)

            combined_df = combined_df.iloc[:,::-1]
            X = combined_df.iloc[:, list(range(self.config.horizon)) + list(range(self.config.horizon*2+1, len(combined_df.columns)-1))]
            y = combined_df.iloc[:, self.config.horizon: self.config.horizon*2]
                
        combined_df.to_csv(os.path.join(self.config.root_dir,"data.csv"),index=False)
        return X,y
            
    def train_test_spliting(self,X,y):

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train.to_csv(os.path.join(self.config.root_dir,"X_train.csv"),index=False)
        X_test.to_csv(os.path.join(self.config.root_dir,"X_test.csv"),index=False)
        y_train.to_csv(os.path.join(self.config.root_dir,"y_train.csv"),index=False)
        y_test.to_csv(os.path.join(self.config.root_dir,"y_test.csv"),index=False)
        
        logger.info("Splited data into training and test  sets")
        logger.info(X_train.shape)
        logger.info(y_train.shape)
        logger.info(X_test.shape)
        logger.info(y_test.shape)
        
        print(X_train.shape)
        print(y_train.shape)
        print(X_test.shape)
        print(y_test.shape)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    X,y = data_transformation.data_windowing()
    data_transformation.train_test_spliting(X,y)
except Exception as e:
    raise e

[2024-04-17 10:13:15,466: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-17 10:13:15,472: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-17 10:13:15,478: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-17 10:13:15,481: INFO: common: created directory at: artifacts]
[2024-04-17 10:13:15,484: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\hp\AppData\Local\Temp\ipykernel_14448\2845361751.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'Y (t+{self.config.horizon-i})'] = data['number_sold'].shift(i)
C:\Users\hp\AppData\Local\Temp\ipykernel_14448\2845361751.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'Y (t+{self.config.horizon-i})'] = data['number_sold'].shift(i)
C:\Users\hp\AppData\Local\Temp\ipykernel_14448\2845361751.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

KeyboardInterrupt: 